In [ ]:
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import multiprocessing as mp
import tqdm

import melanoma

In [ ]:
def resize_image_crop(path, output_path, size=(224, 336)):
    """Original画像のサイズを正規化, 縦横比を維持したままscaling、はみ出し部分をcrop
    Args:
        path (pathlib.Path) : image file path
        size (tuple)        : (height, width)
    """
    img = cv2.imread(str(path))
    h_scale = size[0] / img.shape[0]
    w_scale = size[1] / img.shape[1]
    if h_scale > w_scale:
        scale = h_scale
        height, width = size[0], int(img.shape[1] * h_scale)
    else:
        scale = w_scale
        height, width = int(img.shape[0] * w_scale), size[1]
    resized = cv2.resize(img, (width, height))

    if resized.shape[0] > size[0]:
        top = (resized.shape[0] - size[0]) // 2
        resized = resized[top:top + size[0]]
    if resized.shape[1] > size[1]:
        left = (resized.shape[1] - size[1]) // 2
        resized = resized[:, left:left + size[1]]
    cv2.imwrite(str(output_path), resized)
    return resized

In [ ]:
def resize_image(path, output_path, size=(224, 224)):
    img = cv2.imread(str(path))
    resized = cv2.resize(img, size)
    cv2.imwrite(str(output_path), resized)
    return resized

In [ ]:
pool = mp.Pool(processes=7)
res = []
for path in (melanoma.constants.DATASET_ROOT / "Melanoma").glob("*/Original/*.jpg"):
    output_path = Path(str(path).replace("Original", "Resized").replace("jpg", "png"))
    res.append(pool.apply_async(resize_image, (path, output_path)))
for r in tqdm.tqdm(res):
    r.get()
pool.close()

In [ ]:
file_path = melanoma.constants.DATASET_ROOT /  "Melanoma/train/Original/ISIC_0151200.jpg"
img = cv2.imread(str(file_path))

In [ ]:
img.shape